In [1]:
# E-Commerce Review Intelligence - Metadata Integration
# Notebook 03: Load Product Metadata & Join with Reviews

import os
import findspark

In [2]:
# ============================================================================
# 0. CONFIGURE ENVIRONMENT
# ============================================================================

java_home = r"C:\Program Files\Eclipse Adoptium\jdk-11.0.28.6-hotspot"
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = os.path.join(java_home, 'bin') + os.pathsep + os.environ.get('PATH', '')
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, avg, min, max, first, collect_list,
    when, size, array_contains, explode, split
)
import pandas as pd

print("✓ Environment configured")

✓ Environment configured


In [3]:
# ============================================================================
# 1. INITIALIZE SPARK
# ============================================================================

spark = SparkSession.builder \
    .appName("EcommerceMetadata") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("✓ Spark session created")

✓ Spark session created


In [6]:
import gzip
import json

# Read first few records to see the actual structure
meta_file = r"C:\Users\shafe\OneDrive\Desktop\ecommerce-intelligence\data\raw\meta_Electronics.jsonl.gz"

print("Checking first record for duplicate columns...\n")

with gzip.open(meta_file, 'rt', encoding='utf-8') as f:
    first_record = json.loads(f.readline())
    
    # Check for duplicate keys
    print("All keys in first record:")
    for key in first_record.keys():
        print(f"  - {key}")
    
    # Check if 'alert type' appears
    if 'alert type' in first_record:
        print(f"\n⚠ Found 'alert type': {first_record['alert type']}")

Checking first record for duplicate columns...

All keys in first record:
  - main_category
  - title
  - average_rating
  - rating_number
  - features
  - description
  - price
  - images
  - videos
  - store
  - categories
  - details
  - parent_asin
  - bought_together


In [7]:
# ============================================================================
# 2. LOAD METADATA
# ============================================================================

# Path to your metadata file
metadata_path = r"C:\Users\shafe\OneDrive\Desktop\ecommerce-intelligence\data\raw\meta_Electronics.jsonl.gz"

print(f"\n📂 Loading metadata from: {metadata_path}")

# Check if file exists
if not os.path.exists(metadata_path):
    print(f"\n⚠ Metadata file not found at: {metadata_path}")
    print("\nExpected metadata file locations:")
    print("  - meta_Electronics.jsonl.gz")
    print("  - meta_Electronics.json.gz")
    print("\nPlease verify the filename and update the path above.")
else:
    # Strategy: Load line by line and manually parse to avoid duplicate column issue
    # First, try with multiLine=False to treat each line independently
    try:
        df_meta = spark.read.option("mode", "PERMISSIVE") \
                            .option("columnNameOfCorruptRecord", "_corrupt_record") \
                            .option("multiLine", "false") \
                            .json(metadata_path)
        print("✓ Metadata loaded successfully (standard mode)")
    except Exception as e:
        print(f"⚠ Standard load failed: {str(e)[:100]}")
        print("\nTrying alternative approach: sampling schema from subset...")
        
        # Alternative: Read a small sample to infer schema, then apply it
        # This can help avoid duplicate column issues in malformed records
        from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType
        
        # Define a manual schema for the key columns we care about
        # This bypasses Spark's automatic schema inference which is causing the duplicate
        schema = StructType([
            StructField("parent_asin", StringType(), True),
            StructField("main_category", StringType(), True),
            StructField("title", StringType(), True),
            StructField("average_rating", DoubleType(), True),
            StructField("rating_number", IntegerType(), True),
            StructField("features", ArrayType(StringType()), True),
            StructField("description", ArrayType(StringType()), True),
            StructField("price", StringType(), True),
            StructField("store", StringType(), True),
            StructField("categories", ArrayType(StringType()), True),
            StructField("details", StringType(), True),  # May be dict, we'll parse later
            StructField("bought_together", ArrayType(StringType()), True),
            StructField("images", ArrayType(StringType()), True),
            StructField("videos", ArrayType(StringType()), True),
            StructField("subtitle", StringType(), True),
            StructField("author", StringType(), True),
        ])
        
        # Load with manual schema (ignores extra/problematic columns)
        df_meta = spark.read.option("mode", "PERMISSIVE") \
                            .schema(schema) \
                            .json(metadata_path)
        
        print("✓ Metadata loaded with manual schema (ignored problematic columns)")
    
    print(f"  Total products: {df_meta.count():,}")


📂 Loading metadata from: C:\Users\shafe\OneDrive\Desktop\ecommerce-intelligence\data\raw\meta_Electronics.jsonl.gz
⚠ Standard load failed: [COLUMN_ALREADY_EXISTS] The column `alert type` already exists. Consider to choose another name or r

Trying alternative approach: sampling schema from subset...
✓ Metadata loaded with manual schema (ignored problematic columns)
  Total products: 1,610,012


In [9]:
    # ========================================================================
    # 3. EXPLORE METADATA SCHEMA
    # ========================================================================
    
    print("\n" + "="*80)
    print("METADATA SCHEMA")
    print("="*80)
    
    df_meta.printSchema()
    
    # Check available columns
    print("\nAvailable metadata columns:")
    for col_name in df_meta.columns:
        print(f"  - {col_name}")


METADATA SCHEMA
root
 |-- parent_asin: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- rating_number: integer (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- price: string (nullable = true)
 |-- store: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: string (nullable = true)
 |-- bought_together: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- videos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subtitle: string (nullable = true)
 |-- author: string (nullable = true)


Available metadata columns:
  - parent

In [10]:
# ========================================================================
# 4. METADATA QUALITY CHECK
# ========================================================================

print("\n" + "="*80)
print("METADATA COMPLETENESS")
print("="*80)

total_products = df_meta.count()

print(f"\nTotal products in metadata: {total_products:,}\n")

# Check completeness of key fields
key_fields = ['parent_asin', 'title', 'main_category', 'categories', 
              'price', 'average_rating', 'rating_number', 'features', 
              'description', 'store']

for field in key_fields:
    if field in df_meta.columns:
        non_null = df_meta.filter(col(field).isNotNull()).count()
        pct = (non_null / total_products) * 100
        status = "✓" if pct > 80 else "⚠"
        print(f"  {status} {field:20s}: {pct:6.2f}% ({non_null:,} / {total_products:,})")



METADATA COMPLETENESS

Total products in metadata: 1,610,012

  ✓ parent_asin         : 100.00% (1,610,012 / 1,610,012)
  ✓ title               : 100.00% (1,610,012 / 1,610,012)
  ✓ main_category       :  93.40% (1,503,678 / 1,610,012)
  ✓ categories          : 100.00% (1,610,012 / 1,610,012)
  ⚠ price               :  32.72% (526,765 / 1,610,012)
  ✓ average_rating      : 100.00% (1,610,012 / 1,610,012)
  ✓ rating_number       : 100.00% (1,610,012 / 1,610,012)
  ✓ features            : 100.00% (1,610,012 / 1,610,012)
  ✓ description         : 100.00% (1,610,012 / 1,610,012)
  ✓ store               :  99.41% (1,600,492 / 1,610,012)


In [11]:
# ========================================================================
# 5. SAMPLE METADATA RECORDS
# ========================================================================

print("\n" + "="*80)
print("SAMPLE METADATA RECORDS")
print("="*80)

# Show a few complete records
df_meta.select("parent_asin", "title", "main_category", "price", 
               "average_rating", "rating_number") \
    .show(5, truncate=80, vertical=True)




SAMPLE METADATA RECORDS
-RECORD 0------------------------------------------------------------------------------------------
 parent_asin    | B00MCW7G9M                                                                       
 title          | FS-1051 FATSHARK TELEPORTER V3 HEADSET                                           
 main_category  | All Electronics                                                                  
 price          | NULL                                                                             
 average_rating | 3.5                                                                              
 rating_number  | 6                                                                                
-RECORD 1------------------------------------------------------------------------------------------
 parent_asin    | B00YT6XQSE                                                                       
 title          | Ce-H22B12-S1 4Kx2K Hdmi 4Port                            

In [12]:
# ========================================================================
# 6. LOAD REVIEWS (from previous notebook)
# ========================================================================

print("\n" + "="*80)
print("LOADING REVIEWS")
print("="*80)

reviews_path = r"C:\Users\shafe\OneDrive\Desktop\ecommerce-intelligence\data\raw\electronics_sample_2M.jsonl.gz"

df_reviews = spark.read.json(reviews_path)

print(f"✓ Reviews loaded: {df_reviews.count():,}")


LOADING REVIEWS
✓ Reviews loaded: 2,051,569


In [13]:
# ========================================================================
# 7. FILTER METADATA TO MATCH SAMPLE
# ========================================================================

print("\n" + "="*80)
print("FILTERING METADATA TO SAMPLE PRODUCTS")
print("="*80)

# Get unique products from reviews
sample_products = df_reviews.select("parent_asin").distinct()
sample_count = sample_products.count()

print(f"Unique products in review sample: {sample_count:,}")

# Filter metadata to only include products in our sample
df_meta_sample = df_meta.join(sample_products, on="parent_asin", how="inner")

matched_count = df_meta_sample.count()
print(f"Products with metadata: {matched_count:,}")
print(f"Match rate: {(matched_count/sample_count)*100:.1f}%")

if matched_count < sample_count:
    missing = sample_count - matched_count
    print(f"\n⚠ {missing} products in reviews don't have metadata")


FILTERING METADATA TO SAMPLE PRODUCTS
Unique products in review sample: 47
Products with metadata: 47
Match rate: 100.0%


In [14]:
# ========================================================================
# 8. ANALYZE PRODUCT CATEGORIES
# ========================================================================

print("\n" + "="*80)
print("PRODUCT CATEGORY DISTRIBUTION")
print("="*80)

if 'main_category' in df_meta_sample.columns:
    print("\nTop Categories in Sample:")
    df_meta_sample.groupBy("main_category") \
        .agg(count("*").alias("product_count")) \
        .orderBy(col("product_count").desc()) \
        .show(10, truncate=False)

if 'categories' in df_meta_sample.columns:
    # Categories might be an array - explore structure
    print("\nSample category structures:")
    df_meta_sample.select("parent_asin", "title", "categories") \
        .show(3, truncate=100, vertical=True)



PRODUCT CATEGORY DISTRIBUTION

Top Categories in Sample:
+-------------------------+-------------+
|main_category            |product_count|
+-------------------------+-------------+
|Amazon Devices           |30           |
|All Electronics          |8            |
|Home Audio & Theater     |4            |
|Computers                |2            |
|Camera & Photo           |1            |
|Cell Phones & Accessories|1            |
|Apple Products           |1            |
+-------------------------+-------------+


Sample category structures:
-RECORD 0-----------------------------------------------------------------------------------------------------------
 parent_asin | B07F4P3JH7                                                                                           
 title       | Fire HD 8 Tablet (8" HD Display, 16 GB) - Black (Previous Generation - 8th)                          
 categories  | []                                                                                  

In [15]:
# ========================================================================
# 9. ANALYZE BRANDS/STORES
# ========================================================================

print("\n" + "="*80)
print("BRAND/STORE DISTRIBUTION")
print("="*80)

if 'store' in df_meta_sample.columns:
    print("\nTop Brands/Stores:")
    df_meta_sample.groupBy("store") \
        .agg(count("*").alias("product_count")) \
        .orderBy(col("product_count").desc()) \
        .show(15, truncate=False)



BRAND/STORE DISTRIBUTION

Top Brands/Stores:
+--------------------+-------------+
|store               |product_count|
+--------------------+-------------+
|Amazon              |28           |
|TOZO                |2            |
|WYZE                |1            |
|Nulaxy              |1            |
|DOSS                |1            |
|Apple               |1            |
|Amazon Basics       |1            |
|Anker               |1            |
|SanDisk             |1            |
|JSAUX               |1            |
|Senso               |1            |
|NETGEAR             |1            |
|Cambridge Soundworks|1            |
|Ring                |1            |
|JAMJAKE             |1            |
+--------------------+-------------+
only showing top 15 rows



In [16]:
# ========================================================================
# 10. PRICE ANALYSIS
# ========================================================================

print("\n" + "="*80)
print("PRICE DISTRIBUTION")
print("="*80)

if 'price' in df_meta_sample.columns:
    # Price might be a string like "$29.99" - need to clean it
    price_stats = df_meta_sample.select("price").filter(col("price").isNotNull())
    
    print("\nSample prices (raw format):")
    price_stats.show(10, truncate=False)
    
    # Show price range if numeric
    print("\nNote: Price cleaning/parsing will be done in transformation pipeline")


PRICE DISTRIBUTION

Sample prices (raw format):
+-----+
|price|
+-----+
|39.99|
|19.99|
|48.29|
|79.99|
|8.55 |
|29.99|
|49.99|
|99.0 |
|11.99|
|35.99|
+-----+
only showing top 10 rows


Note: Price cleaning/parsing will be done in transformation pipeline


In [17]:
# ========================================================================
# 11. JOIN REVIEWS WITH METADATA
# ========================================================================

print("\n" + "="*80)
print("CREATING ENRICHED DATASET")
print("="*80)

# IMPORTANT: Both datasets have 'title' column - need to disambiguate
# Review title = short summary by reviewer ("Great product!")
# Product title = official product name ("Amazon Echo Dot 4th Gen")

# Rename columns before join to avoid collision
df_meta_renamed = df_meta_sample.select(
    col("parent_asin"),
    col("title").alias("product_title"),  # Product name
    col("main_category"),
    col("store"),
    col("price"),
    col("features"),
    col("description")
)

# Join reviews with metadata
df_enriched = df_reviews.join(
    df_meta_renamed,
    on="parent_asin",
    how="left"
)

# Rename review title for clarity
df_enriched = df_enriched.withColumnRenamed("title", "review_title")

print(f"✓ Created enriched dataset")
print(f"  Total reviews: {df_enriched.count():,}")
print(f"  Columns: {len(df_enriched.columns)}")

# Verify we have both titles
print(f"\nColumn check:")
print(f"  - review_title: {'✓' if 'review_title' in df_enriched.columns else '✗'}")
print(f"  - product_title: {'✓' if 'product_title' in df_enriched.columns else '✗'}")

# Sample enriched records
print("\nSample Enriched Records (Review + Product Info):")
df_enriched.select("rating", "review_title", "product_title", "store", "main_category") \
    .show(3, truncate=100, vertical=True)



CREATING ENRICHED DATASET
✓ Created enriched dataset
  Total reviews: 2,051,569
  Columns: 16

Column check:
  - review_title: ✓
  - product_title: ✓

Sample Enriched Records (Review + Product Info):
-RECORD 0-------------------------------------------------------------------------------------------------------------
 rating        | 5.0                                                                                                  
 review_title  | Excellent!                                                                                           
 product_title | Senso Bluetooth Headphones, Best Wireless Sports Earbuds w/Mic IPX7 Waterproof HD Stereo Sweatpro... 
 store         | Senso                                                                                                
 main_category | All Electronics                                                                                      
-RECORD 1----------------------------------------------------------------------------

In [18]:
# ========================================================================
# 12. COMPETITIVE ANALYSIS PREVIEW
# ========================================================================

print("\n" + "="*80)
print("COMPETITIVE INTELLIGENCE PREVIEW")
print("="*80)

# Group by store/brand with review stats
if 'store' in df_enriched.columns:
    brand_performance = df_enriched.groupBy("store") \
        .agg(
            count("*").alias("total_reviews"),
            avg("rating").alias("avg_rating"),
            count(when(col("rating") >= 4, 1)).alias("positive_reviews"),
            count(when(col("rating") <= 2, 1)).alias("negative_reviews")
        ) \
        .withColumn(
            "satisfaction_rate",
            (col("positive_reviews") / col("total_reviews") * 100)
        ) \
        .orderBy(col("total_reviews").desc())
    
    print("\nBrand Performance Summary (Top 15):")
    brand_performance.show(15, truncate=False)


COMPETITIVE INTELLIGENCE PREVIEW

Brand Performance Summary (Top 15):
+--------------------+-------------+------------------+----------------+----------------+-----------------+
|store               |total_reviews|avg_rating        |positive_reviews|negative_reviews|satisfaction_rate|
+--------------------+-------------+------------------+----------------+----------------+-----------------+
|Amazon              |1454533      |4.33138264996394  |1195292         |168370          |82.17702864080773|
|TOZO                |116029       |4.2712942454041665|93467           |15431           |80.55486128467882|
|Panasonic           |55743        |4.255404266006494 |44810           |6927            |80.38677502107888|
|Cambridge Soundworks|51568        |4.3570819112627985|42880           |5922            |83.15234253800806|
|Apple               |30538        |4.356179186587203 |25267           |4018            |82.73953762525377|
|Cheetah             |29960        |4.4663217623498   |25959     

In [19]:
# ========================================================================
# 13. SAVE METADATA SAMPLE
# ========================================================================

print("\n" + "="*80)
print("SAVING METADATA SAMPLE")
print("="*80)

# Cache for future use
df_meta_sample.cache()
df_enriched.cache()

output_path = r"C:\Users\shafe\OneDrive\Desktop\ecommerce-intelligence\data\processed"

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

print(f"✓ Datasets cached in memory")
print(f"  - df_meta_sample: {matched_count:,} products")
print(f"  - df_enriched: {df_enriched.count():,} reviews with metadata")


SAVING METADATA SAMPLE
✓ Datasets cached in memory
  - df_meta_sample: 47 products
  - df_enriched: 2,051,569 reviews with metadata


In [20]:
# ========================================================================
# 14. KEY INSIGHTS FOR FEATURE DEVELOPMENT
# ========================================================================

print("\n" + "="*80)
print("INSIGHTS FOR FEATURE DEVELOPMENT")
print("="*80)

print("""
✓ METADATA INTEGRATION COMPLETE

NOW AVAILABLE FOR FEATURES:

1. SENTIMENT TRENDS (Enhanced)
   → Can now track sentiment by brand, category, price tier
   → Compare sentiment across competitor products

2. THEME EXTRACTION (Enhanced)
   → Extract themes specific to product categories
   → Compare themes across brands (e.g., "battery life" for Apple vs competitors)

3. COMPETITIVE ANALYSIS (Core Feature)
   → Brand-level performance metrics
   → Category-level comparisons
   → Price tier analysis
   → Feature-based competitive positioning

4. PRODUCT SUCCESS PREDICTION (Enhanced)
   → Use product attributes (category, price, brand) as features
   → Predict success based on similar product patterns

5. REVIEW VELOCITY (Enhanced)
   → Velocity by brand/category
   → Seasonal patterns by product type

NEXT STEPS:
1. Clean price data (convert "$29.99" to numeric)
2. Parse category hierarchies if nested
3. Extract key product features from metadata
4. Build transformation pipeline with enriched data
5. Design Redshift schema including product dimensions
""")

print("\n🎯 Ready to build enriched transformation pipeline!")
print(f"   Access enriched data via: df_enriched")
print(f"   Access metadata via: df_meta_sample")


INSIGHTS FOR FEATURE DEVELOPMENT

✓ METADATA INTEGRATION COMPLETE

NOW AVAILABLE FOR FEATURES:

1. SENTIMENT TRENDS (Enhanced)
   → Can now track sentiment by brand, category, price tier
   → Compare sentiment across competitor products

2. THEME EXTRACTION (Enhanced)
   → Extract themes specific to product categories
   → Compare themes across brands (e.g., "battery life" for Apple vs competitors)

3. COMPETITIVE ANALYSIS (Core Feature)
   → Brand-level performance metrics
   → Category-level comparisons
   → Price tier analysis
   → Feature-based competitive positioning

4. PRODUCT SUCCESS PREDICTION (Enhanced)
   → Use product attributes (category, price, brand) as features
   → Predict success based on similar product patterns

5. REVIEW VELOCITY (Enhanced)
   → Velocity by brand/category
   → Seasonal patterns by product type

NEXT STEPS:
1. Clean price data (convert "$29.99" to numeric)
2. Parse category hierarchies if nested
3. Extract key product features from metadata
4. Buil